In [ ]:
import json
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
import boto3
from typing import List, Dict

def get_taxi_data(formatted_datetime: str) -> Dict:
    """
    Retrieves the data for a given date

    Parameters:
        formatted_datetime (str): The date in "YYYY-MM-DD" format.

    Returns:
        Dict: a dictionary containing the taxi data in Json.

    """

    taxi_url = (
    f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' "
    f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000")

    response_taxi = requests.get(taxi_url)
    taxi_data = response_taxi.json()

    return taxi_data

def get_weather_data(formatted_datetime) -> Dict:
    weather_url = "https://archive-api.open-meteo.com/v1/era5"

    #date = "2013-06-01"

    params = {
    "latitude" : 41.85, 
    "longitude" : -87.65,
    "start_date" : formatted_datetime,
    "end_date" : formatted_datetime,
    "hourly" : "temperature_2m,wind_speed_10m,rain,precipitation"
    }

    response_weather = requests.get(weather_url, params=params)
    weather_data = response_weather.json()

    return weather_data

def upload_to_s3(data: Dict, folder_name: str, file_name: str) -> None:
    client_taxi = boto3.client('s3')
    client_taxi.put_object(
    Bucket="cubix-taxi-data-tm",
    Key=f"raw_data/to_process/{folder_name}/{file_name}",
    Body=json.dumps(data))


def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=2)
    formatted_datetime = current_datetime.strftime("%Y-%m-%d")

    taxi_data_api_call =get_taxi_data(formatted_datetime)
    weather_data_api_call = get_weather_data(formatted_datetime)

    taxi_file_name = f"taxi_raw{formatted_datetime}.json"
    weather_file_name = f"weather_raw{formatted_datetime}.json"

    upload_to_s3(data=taxi_data_api_call, file_name=taxi_file_name, folder_name="taxi_data")
    upload_to_s3(data=weather_data_api_call, file_name=weather_file_name, folder_name="weather_data")